In [1]:
JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"

RANDOM_SEED = 42 # for reproducibility

# Preparazione

In [5]:
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=1be3368d549688a023c68e0ebefc74160833b01b86ad86c530b0060576c21f92
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [ ]:
!ngrok authtoken 2Tn9uykXlYRjLcSLIgmv1iLdEvG_3gqA9M3V4hLGqH9N3JWEr

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [2]:
!apt install openjdk-8-jdk-headless -qq

The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 16 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 120831 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headless_8u382-ga-1~22.04.1_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u382-ga-1~22.04.1) ...

In [6]:
from pyngrok import ngrok
import os
import re

In [7]:
os.environ["JAVA_HOME"] = JAVA_HOME

In [8]:
# Connect this colab to Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
# Install PySpark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=905c4f687c582d4ee49a107f096e72a23d7a897aca3642b03f352b7516dc0941
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [10]:
# Import libraries
# POI CAPISCI QUALE TI SERVE VERAMENTE
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql import functions as sqlf
from pyspark import SparkContext, SparkConf
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from nltk.stem.snowball import SnowballStemmer
from pyspark.ml.feature import HashingTF, CountVectorizer, IDF
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import Normalizer
from pyspark.ml.evaluation import ClusteringEvaluator
import gc

In [11]:
# Create the session
# POI CAPISCI SE SONO I PARAMETRI GIUSTI


# Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '4G').\
                set('spark.driver.memory', '45G').\
                set('spark.driver.maxResultSize', '10G').\
                set('setCheckpointDir', '/content/gdrive/MyDrive/VideogameRecommenderSystem/checkpoints_pyspark').\
                setAppName("VideogameRecommenderSystem").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [12]:
port = '4050'
public_url = ngrok.connect(port).public_url

In [13]:
print("To access the Spark Web UI console, please click on the following link to the ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

To access the Spark Web UI console, please click on the following link to the ngrok tunnel "https://856d-34-74-132-107.ngrok.io" -> "http://127.0.0.1:4050"


In [14]:
spark

In [15]:
games_df = spark.read.load('/content/gdrive/MyDrive/VideogameRecommenderSystem/games.csv',
                           format="csv",
                           sep=",",
                           inferSchema="true",
                           header="true")

In [16]:
print("Row number:", games_df.count(), "\nColumn number:", len(games_df.columns))

Row number: 131974 
Column number: 6


In [18]:
# Remove the links from the games descriptions since they are not relevant for
# the project
CLEANER_LINKS = re.compile('http[s]?://\S+')

def clean_links(raw_text):
  cleantext = re.sub(CLEANER_LINKS, '', raw_text)
  return cleantext

In [41]:
# SPIEGA PERCHÉ HAI BISOGNO DI STO UDF
clean_links_UDF = sqlf.udf(lambda x : clean_links(x), StringType())

In [25]:
CLEANER_PUNCTUATION = re.compile('[^\w\s]')

def clean_punctuation(raw_text):
  cleantext = re.sub(CLEANER_PUNCTUATION, ' ', raw_text)
  return cleantext

In [26]:
clean_punctuation_UDF = sqlf.udf(lambda x : clean_punctuation(x), StringType())

In [42]:
CLEANER_EXTRA_SPACES = re.compile('\s\s+')

def clean_extra_spaces(raw_text):
  cleantext = re.sub(CLEANER_EXTRA_SPACES, ' ', raw_text)
  return cleantext

In [29]:
clean_extra_spaces_UDF = sqlf.udf(lambda x : clean_extra_spaces(x), StringType())

In [ ]:
@sqlf.udf("long")
def num_nonzeros(v):
  return v.numNonzeros()

In [ ]:
def process_data_for_clustering(dataset_df,
                                column_name):
  dataset_df = dataset_df.dropna()
  dataset_df = dataset_df.withColumn(column_name, clean_links_UDF(sqlf.col(column_name)))
  dataset_df = dataset_df.withColumn(column_name, sqlf.trim(sqlf.col(column_name)))

  dataset_df = dataset_df.where(dataset_df.total_recommendations > 0)

  dataset_df = dataset_df.withColumn(column_name, sqlf.lower(sqlf.col(column_name)))
  dataset_df = dataset_df.withColumn(column_name, clean_punctuation_UDF(sqlf.col(column_name)))
  dataset_df = dataset_df.withColumn(column_name, clean_extra_spaces_UDF(sqlf.col(column_name)))

  tokenizer = Tokenizer(inputCol=column_name, outputCol='tokens')
  tokens_df = tokenizer.transform(dataset_df)

  stopwords_remover = StopWordsRemover(inputCol='tokens', outputCol='terms')
  terms_df = stopwords_remover.transform(tokens_df)

  stemmer = SnowballStemmer(language="english")
  stemmer_udf = sqlf.udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))
  terms_stemmed_df = terms_df.withColumn('terms_stemmed', stemmer_udf('terms'))

  cv = CountVectorizer(inputCol='terms_stemmed', outputCol="tf_features", vocabSize=1000, minDF=10)
  idf = IDF(inputCol="tf_features", outputCol="features")

  pipeline = Pipeline(stages=[cv, idf])
  features = pipeline.fit(terms_stemmed_df)
  tf_idf_df = features.transform(terms_stemmed_df)

  tf_idf_df = tf_idf_df.where(num_nonzeros("features") > 0)

  return tf_idf_df

In [ ]:
tf_idf_df = process_data_for_clustering(games_df, 'about_the_game')

In [ ]:
#tf_idf_df.count()

# Funzioni

In [ ]:
def k_means(dataset,
            n_clusters,
            features_col='features',
            prediction_col='cluster',
            random_seed=RANDOM_SEED):

  print("""K-means parameter:
  - K (number of clusters) = {:d}
  - random seed = {:d}
  """.format(n_clusters, random_seed))

  dataset = Normalizer(inputCol=features_col, outputCol=features_col+"_norm", p=1).transform(dataset)
  features_col = features_col+"_norm"

  k_means = KMeans(
      featuresCol=features_col,
      predictionCol=prediction_col,
      k=n_clusters,
      initMode='k-means||',
      initSteps=5,
      tol=0.000001,
      maxIter=20,
      seed=random_seed,
      distanceMeasure='cosine'
      )
  model = k_means.fit(dataset)

  clusters_df = model.transform(dataset)

  return model, clusters_df

In [ ]:
def evaluate_k_means(clusters,
                     metric_name="silhouette",
                     distance_measure="cosine",
                     prediction_col="cluster"
                     ):

  evaluator = ClusteringEvaluator(metricName=metric_name,
                                  distanceMeasure=distance_measure,
                                  predictionCol=prediction_col
                                  )

  return evaluator.evaluate(clusters)

# Clustering

In [ ]:
k = 100
print("Running K-means using K = {:d}".format(k))
model, clusters_df = k_means(tf_idf_df, k)
silhouette_k = evaluate_k_means(clusters_df)
wssd_k = model.summary.trainingCost
print("Silhouette coefficient computed with cosine distance: {:.3f}".format(silhouette_k))
print("Within-cluster Sum of Squared Distances (using cosine distance): {:.3f}".format(wssd_k))
print("--------------------------------------------------------------------------------------")

In [ ]:
path = '/content/gdrive/MyDrive/VideogameRecommenderSystem/models/KMEANS_about_K_{}'.format(k)
model.save(path)